In [109]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
import numpy as np

In [110]:
EMBEDDING_DIMENSIONALITY = 512

In [111]:
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [112]:
query = "I just discovered the course. Can I join now?"

In [113]:
embeddings_q = list(model.embed(query))[0]

In [114]:
np.min(embeddings_q)

np.float64(-0.11726373551188797)

In [115]:
doc = 'Can I still join the course after the start date?'

In [116]:
embeddings_d = list(model.embed(doc))[0]

In [117]:
len(embeddings_d)

512

In [118]:
# Since it's already normalized, we can just use dot product to get the cosine similarity

embeddings_q.dot(embeddings_d)

np.float64(0.9008528856818036)

In [119]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [120]:
V = np.zeros((5, 512))

for i, doc in enumerate(documents):
    embeddings_d = list(model.embed(doc['text']))[0]
    V[i] = embeddings_d

In [121]:
max_index = np.argmax(V.dot(embeddings_q))
max_index

np.int64(1)

In [122]:
V = np.zeros((5, 512))

for i, doc in enumerate(documents):
    full_text = doc['question'] + ' ' + doc['text']
    embeddings_d = list(model.embed(full_text))[0]
    V[i] = embeddings_d

max_index = np.argmax(V.dot(embeddings_q))
max_index

np.int64(0)

In [123]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [124]:
EMBEDDING_DIMENSIONALITY = 384
model_handle = "BAAI/bge-small-en"

In [125]:
from qdrant_client import QdrantClient, models

qd_client = QdrantClient("http://localhost:6333")

In [126]:
collection_name = "zoomcamp-hw2"

In [127]:
qd_client.delete_collection(collection_name=collection_name)

True

In [128]:
# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY, 
        distance=models.Distance.COSINE
    )
)

True

In [129]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [130]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [131]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [134]:
def vector_search(question):
    print("Vector search is used")
    # course = "data-engineering-zoomcamp"
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        limit=5,
        with_payload=True
    )

    results = []

    for point in query_points.points:
        results.append(point)

    return results

In [135]:
vector_search(query)

Vector search is used


[ScoredPoint(id=14, version=2, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=6, version=2, score=0.8691884, payload={'text': 'Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)', 'section': 'General course-related questions', 'question': 'How long is the course?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=Non